Import Libraries

In [ ]:
import urllib
import scipy
from bs4 import *
import requests
from bs4 import BeautifulSoup
import os
import lxml
import html5lib
import numpy as np
#!pip install simplejson
import simplejson

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#!pip install selenium
#!apt-get update # to update ubuntu to correctly run apt install
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin
from selenium import webdriver



In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')



#Web Scraping

In [ ]:
rootdir= "/content/drive/MyDrive/SpeakerRecognition"

In [ ]:
#import os
#os.mkdir(rootdir+"/audio")

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

names = []
temp_name = ""
temp_size = ""
counter = 1
links = []

In [ ]:
# with urllib.request.urlopen("https://librivox.org/search?primary_key=0&search_category=title&search_page=1&search_form=get_results") as url:
#     html = url.read()


for i in range(190, 300):
    print(i)
    url = ("https://librivox.org/search?title=&author=&reader=&keywords=&genre_id=0&status=all&project_type=solo&recorded_language=1&sort_order=catalog_date&search_page={}&search_form=advanced").format(i)
    
    browser = webdriver.Chrome('chromedriver',options=options)
    browser.get(url)

    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME , "catalog-result"))
    )

    html = browser.page_source
    soup = BeautifulSoup(html, 'html5lib')    

    
    for litag in soup.find_all('li', {'class': 'catalog-result'}):
        h = litag.find('h3')
        browser.get(h.a['href'])
        html2 = browser.page_source
        soup2 = BeautifulSoup(html2, 'html5lib')

        s = soup2.find('dl', {'class': 'product-details'})
            
        target =  s.find_all('dd')
        
        temp_name = target[3].get_text()
        temp_size = target[1].get_text()
        temp_size = temp_size.replace('MB','')

        if(temp_name in names):
            continue

        s = soup2.find('dl', {'class': 'listen-download'})
        
        target = s.find_all('dd')
        #print(temp_size)
        #print(type(temp_size)=="<class 'str'>")
        #if(isinstance(temp_size,str)):
         # continue
        try:
          if float(temp_size) >= 25 and float(temp_size) <= 110:
            links.append(target[0].a['href'])
            names.append(temp_name)
            print(temp_name,temp_size)
        except ValueError:
          continue
          
       ## Store at each 10 iterations   
    if(i%10==0):
         f = open(rootdir+'/book_links.txt', 'w')
         simplejson.dump(links, f)
         f.close()

         f = open(rootdir+'/author_names.txt', 'w')
         simplejson.dump(names, f)
         f.close()
         print("New speakers stored")
                                
#             for dtag in litag.find_all('dt'):
#             #                 print(dtag.span.text)
#             #                 print(dtag.a['href'])
#                 x = dtag.span.text
#                 x = x.replace('MB','')
#                 if float(x) >= 25 and float(x) <= 110:
#                         links.append(dtag.a['href'])
    browser.quit()
    print(len(names)," speakers gathered.\n")
#print(links)
#print(names)
    #for link in links:
    #    localDestination = rootdir+"/audio/audio{}.wav".format(counter)
    #    resultFilePath, responseHeaders = urllib.request.urlretrieve(link, localDestination)
    #    counter += 1
        
## Start at page 39

In [ ]:
f = open(rootdir+'/book_links.txt', 'w')
simplejson.dump(links, f)
f.close()

f = open(rootdir+'/author_names.txt', 'w')
simplejson.dump(names, f)
f.close()


In [ ]:
links

In [ ]:
names

## Downloading data for 200 speakers

In [2]:
rootdir= "/content/drive/My Drive/SpeakerRecognition"

In [27]:
import random
import numpy as np
import urllib
import os

In [4]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [37]:
if (not os.path.exists(rootdir+"/audio")):
  os.mkdir(rootdir+"/audio")

with open(rootdir+'/book_links.txt') as f:
    links = f.readlines()

links=[element for element in links ]
links=links[0][1:-1].replace('"','').split(", ")

counter=1


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
